# **PROJECT- GOURMIND**

> ### Import prophet time series algorithm by facebook

In [1]:
!python -m pip install prophet

> ### importing libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from prophet import Prophet
import os

Matplotlib is building the font cache; this may take a moment.


ModuleNotFoundError: No module named 'prophet'

In [ ]:
path = os.path.join(os.getcwd(), "ICRISAT-District Level Data.csv")
print(path)
agri_df = pd.read_csv(path)

In [ ]:
agri_df.drop(agri_df.iloc[:, -6:], inplace=True, axis=1)
agri_df.head()

In [5]:
agri_df.isnull().sum()


Dist Code                           0
Year                                0
State Code                          0
State Name                          0
Dist Name                           0
                                   ..
SUGARCANE PRODUCTION (1000 tons)    0
SUGARCANE YIELD (Kg per ha)         0
COTTON AREA (1000 ha)               0
COTTON PRODUCTION (1000 tons)       0
COTTON YIELD (Kg per ha)            0
Length: 74, dtype: int64

In [6]:
agri_df.rename(columns = {'State Code':'state_code', 'State Name':'state',
                              'Dist Name':'dist'}, inplace = True)
print(agri_df.columns[:20])

Index(['Dist Code', 'Year', 'state_code', 'state', 'dist',
       'RICE AREA (1000 ha)', 'RICE PRODUCTION (1000 tons)',
       'RICE YIELD (Kg per ha)', 'WHEAT AREA (1000 ha)',
       'WHEAT PRODUCTION (1000 tons)', 'WHEAT YIELD (Kg per ha)',
       'KHARIF SORGHUM AREA (1000 ha)',
       'KHARIF SORGHUM PRODUCTION (1000 tons)',
       'KHARIF SORGHUM YIELD (Kg per ha)', 'RABI SORGHUM AREA (1000 ha)',
       'RABI SORGHUM PRODUCTION (1000 tons)', 'RABI SORGHUM YIELD (Kg per ha)',
       'SORGHUM AREA (1000 ha)', 'SORGHUM PRODUCTION (1000 tons)',
       'SORGHUM YIELD (Kg per ha)'],
      dtype='object')


In [7]:
d= {}
for i in agri_df.columns:
    d[i]=("_".join(i.split()[:2])).lower()
    
agri_df.rename(columns = d, inplace = True)
print(agri_df.columns[:20])

Index(['dist_code', 'year', 'state_code', 'state', 'dist', 'rice_area',
       'rice_production', 'rice_yield', 'wheat_area', 'wheat_production',
       'wheat_yield', 'kharif_sorghum', 'kharif_sorghum', 'kharif_sorghum',
       'rabi_sorghum', 'rabi_sorghum', 'rabi_sorghum', 'sorghum_area',
       'sorghum_production', 'sorghum_yield'],
      dtype='object')


In [8]:
ka_df = agri_df[agri_df["state"]=="Karnataka"]
print(np.unique(ka_df["dist"]))
# bang = ka_df[ka_df["dist"] == "Bangalore"]
# bang.columns

['Bangalore' 'Belgaum' 'Bellary' 'Bidar' 'Bijapur / Vijayapura'
 'Chickmagalur' 'Chitradurga' 'Dakshina Kannada' 'Dharwad'
 'Gulbarga / Kalaburagi' 'Hassan' 'Kodagu / Coorg' 'Kolar' 'Mandya'
 'Mysore' 'Raichur' 'Shimoge' 'Tumkur' 'Uttara Kannada']


In [9]:
ka_df = ka_df.replace(to_replace=["Bijapur / Vijayapura", "Gulbarga / Kalaburagi", "Kodagu / Coorg"], value=["Vijaypura", "Gulbarga", "Coorg"])

In [10]:
print(np.unique(ka_df["dist"]))

['Bangalore' 'Belgaum' 'Bellary' 'Bidar' 'Chickmagalur' 'Chitradurga'
 'Coorg' 'Dakshina Kannada' 'Dharwad' 'Gulbarga' 'Hassan' 'Kolar' 'Mandya'
 'Mysore' 'Raichur' 'Shimoge' 'Tumkur' 'Uttara Kannada' 'Vijaypura']


In [ ]:
class Crop(object):
    def __init__(self, dist, crop):
        self.dist = dist
        self.crop = crop
        self.dist_data = ka_df[ka_df["dist"] == self.dist]
        self.pp_crop = [x for x in self.dist_data.iloc[:, 5:].columns if x.startswith(self.crop)]
        
    def plotter(self):
        fig, axe = plt.subplots(3, figsize=(10, 10))
        sns.lineplot(x="year", y=self.pp_crop[0], data=self.dist_data, color="r", ax=axe[0])
        sns.lineplot(x="year", y=self.pp_crop[1], data=self.dist_data, color="g", ax=axe[1])
        sns.lineplot(x="year", y=self.pp_crop[2], data=self.dist_data, color="b", ax=axe[2])
        fig.tight_layout(pad = 1.5)
        plt.savefig('/tmp/plot.png')
        
    def pp_model(self):
        m = Prophet()
        df = self.dist_data.loc[:, ["year", self.pp_crop[-1]]]
        cols = df.columns
        df.rename(columns={cols[0]: "ds", cols[1]: "y"}, inplace=True)
        m.fit(df)
        future = m.make_future_dataframe(periods=365)
        forecast = m.predict(future)
        fig1 = m.plot(forecast)
        

In [ ]:
district = input("Enter the district of Karnataka:").strip().capitalize()
crop_name = input("Enter the crop:").strip().lower()
dataplot = Crop(district, crop_name)
dataplot.plotter()

In [ ]:
np.unique(ka_df.columns)

In [ ]:
plt.figure(figsize=(25,15))
sns.barplot(x=X, y=y)
plt.xticks(rotation=90)
plt.show()

In [ ]:
plt.tick_params(labelsize=10)
sns.heatmap(agri_df[["crop_year", "area", "production"]].corr(),annot=True)
plt.show()

In [ ]:
dataplot.pp_model()

In [5]:
import json
import requests

# The API endpoint for your Hub API project
endpoint = f"http://localhost:9000/2015-03-31/functions/function/invocations"
API_KEY = "sDeaNMxtOs6SxxBR4ZREJai4cxyRCI6s8HKyD2B9"

headers = {
  "Content-Type": "application/json",
    "x-api-key": API_KEY
}

payload = {
    "district": "Bangalore",
    "crop_name": "rice"
}

# Send the POST request
response = requests.post(endpoint, headers=headers, data=json.dumps(payload)).json()

if response.get('statusCode') == 200:
    output = response['body']['output']
else:
    output = response

print(output)

ConnectionError: HTTPConnectionPool(host='localhost', port=9000): Max retries exceeded with url: /2015-03-31/functions/function/invocations (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f2c4ae54790>: Failed to establish a new connection: [Errno 111] Connection refused'))